In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
torch.set_default_tensor_type(torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor)

In [3]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=(3,3))
        self.conv2 = nn.Conv2d(64, 64, kernel_size=(3,5))
        self.fc1 = nn.Linear(12544, 32)
        self.fc2 = nn.Linear(32, 10)
        self.dropout = nn.Dropout(.2)
    def forward(self, x):
        x = x.view(-1, 1, 64, 256)
        x = F.relu(self.conv1(x))
        x = self.dropout(x)
        x = F.max_pool2d(x, (2,4))
        x = F.relu(self.conv2(x))
        x = self.dropout(x)
        x = F.max_pool2d(x, (2,4))
        x = x.view(-1, 12544)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        return F.softmax(x, dim=1)

In [4]:
def assess(model, songs, labels):
    correct = 0
    total = 0
    for i in range(50):
        index = np.random.randint(len(labels))
        pred = model(songs[index])
        if torch.argmax(pred, dim=1) == labels[index]:
            correct += 1
        total += 1
    return correct/total

In [6]:
with np.load("../audio_sr_label.npz") as f:
    data = f['X']
    labels = list(f['T'])
data = np.asarray([[i[:2560] for i in j] for j in data])
seed = data[:, :, :256]
for i in range(1, 10):
    seed = np.append(seed, data[:, :, i*256:(i+1)*256], axis=0)
data = torch.tensor(seed, dtype=torch.float)
labels = labels * 10 #expand dimensions accordingly
label_set = set(labels)
mapping = {}
for count, i in enumerate(label_set):
    mapping[i] = count
targets = np.zeros(len(labels))
for i in range(len(targets)):
    targets[i] = mapping[labels[i]]
targets = torch.tensor(targets, dtype=torch.long)

In [7]:
indices = np.random.choice(np.arange(len(targets)), len(targets), replace=False)
train_index = indices[int(len(indices)/10):]
test_index = indices[:int(len(indices)/10)]

labels_train = targets[train_index]
labels_test = targets[test_index]
data_train = data[train_index]
data_test = data[test_index]
#double check ^^ above stuff

In [10]:
model = Model()
optimizer = optim.Adam(list(model.parameters()), lr=1e-5)
if torch.cuda.is_available():
    model.cuda()
    data_train.cuda()
    data_test.cuda()
    labels_train.cuda()
    labels_test.cuda()

In [11]:
training_acc = []
validation_acc = []
model.train()
epochs = 50
for epoch in range(epochs):
    for i in range(len(labels_train)):
        index = np.random.randint(len(labels_train))
        optimizer.zero_grad()
        target = labels_train[index]
        prediction = model(data_train[index])
        criterion = nn.CrossEntropyLoss()
        loss = criterion(prediction, target.unsqueeze(0))
        loss.backward()
        optimizer.step()
#         if i % 200 == 0:
#             training_acc.append(assess(model, data_train, labels_train))
#             validation_acc.append(assess(model, data_test, labels_test))
        if i % 1000 == 0:
            train_checkpoint = assess(model, data_train, labels_train)
            model.eval()
            test_checkpoint = assess(model, data_test, labels_test)
            model.train()
            print("loss: ", loss)
            print(train_checkpoint, test_checkpoint)
#                 with open("training_acc", "ab") as f:
#                     np.asarray(training_acc).tofile(f)
#                 with open("validation_acc", "ab") as f:
#                     np.asarray(validation_acc).tofile(f)
#                 training_acc = []
#                 validation_acc = []
    torch.save(model.state_dict(), "./model" + str(epoch).zfill(2))

loss:  tensor(2.2830, grad_fn=<NllLossBackward>)
0.08 0.12


KeyboardInterrupt: 

In [14]:
for epoch in range(epochs):
    model = Model()
    model.load_state_dict(torch.load("./model" + str(epoch).zfill(2)))
    #model.load_state_dict(torch.load("./model", map_location='cpu'))
    model.eval()
    correct = 0
    total = 0
    for i in range(int(len(data)/10)):
        y_hats = model(data[i])
        for j in range(1, 10):
            y_hats = torch.cat((y_hats, model(data[i + 1000*j])))
        if torch.argmax(torch.sum(y_hats, dim=0)) == targets[i]:
            correct += 1
        total += 1
    print(epoch, ": ", correct / total)

0 :  0.021
1 :  0.021
2 :  0.021


KeyboardInterrupt: 